In [1]:
import textwrap, sys, os, glob, shutil
import numpy as np
import MDAnalysis as mda
import mdtraj as md
from pdbfixer import PDBFixer
from openbabel import openbabel
from datetime import datetime
#OpenFF
import openff
import openff.units
import openff.toolkit
import openff.interchange
#OpenMM
from openmm.app import *
from openmm import *
from openmm.unit import *
#Custom
from ForceFieldHandler import *
from OpenMMJoiner import *
from ProteinPreparer import *
from Initter import *
from bp_utils import *

## Steps to Prepare

In [2]:
start = datetime.now()

In [3]:
job_inputs = {'working_dir_name': 'gpcr_join_testing',
              'ligand_resname': 'resname V4O',
              'crystal_pdb_fn': '5c1m_V4O.pdb',
              'build_modes': ['MEM', 'SOL']}

In [4]:
working_dir_name = job_inputs['working_dir_name']
ligand_resname = job_inputs['ligand_resname']
crystal_pdb_fn = job_inputs['crystal_pdb_fn']

In [5]:
receptor_path, ligand_sdf_path = Initter(crystal_pdb_fn, ligand_resname, working_dir_name).main()

/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/exouser/miniconda3/envs/openff/lib/python3.12/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys.pdb True
66 Atoms 72 Bonds 1 Residues
66 Atoms 72 Bonds 1 Residues
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys_H.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/ligand_crys.sdf True


In [6]:
protein_solvated = ProteinPreparer(receptor_path).main()

Protanting using command line
Running pdb2pqr30 --ff AMBER --nodebump --keep-chain --ffout AMBER --pdb-output /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pdb --with-ph 7 /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pqr


INFO:PDB2PQR v3.6.2: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     1/2 of bilayer thickness:   17.2>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 413 residues and 3249 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:Attempting to repair 16 mis

Done with exit status 0
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pdb True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H.pqr True
/media/volume/sdb/githubs/Bridgeport/gpcr_join_testing/receptor_crys_H_MEM.pdb True


In [7]:
ligand_sys, ligand_top, ligand_positions = ForceFieldHandler(ligand_sdf_path).main()

/media/volume/sdb/githubs/Bridgeport/ForceFieldHandler.py:82: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  positions = np.array(interchange.positions) * nanometer


In [8]:
receptor_sys, receptor_top, receptor_positions = ForceFieldHandler(protein_solvated).main()

In [9]:
comp_sys, comp_top, comp_positions = Joiner((ligand_sys, ligand_top, ligand_positions),
                                            (receptor_sys, receptor_top, receptor_positions)).main()

In [10]:
box_vectors = comp_sys.getDefaultPeriodicBoxVectors()

translate = Quantity(np.array((box_vectors[0].x,
                               box_vectors[1].y,
                               box_vectors[2].z))/2,
                     unit=nanometer)

In [11]:
integrator = LangevinIntegrator(300 * kelvin, 1/picosecond, 0.001 * picosecond)

In [12]:
simulation = Simulation(comp_top, comp_sys, integrator)
simulation.context.setPositions(comp_positions + translate)

In [13]:
#Evaluate and Report pre-minimized energy
describe_state(simulation.context.getState(getEnergy=True, getForces=True), "Original state")
end = datetime.now() - start
print(f'Time to build this simulation: {end}')

Original state has energy -1799027.22 kJ/mol  with maximum force 8071221.57 kJ/(mol nm)
Time to build this simulation: 0:03:42.935349


In [14]:
#Write Out (and try minimizing on a GPU)
with open('result.pdb', 'w') as f:
    PDBFile.writeFile(simulation.topology, simulation.context.getState(getPositions=True).getPositions(), f)
    
with open('result.xml', 'w') as f:
    f.write(XmlSerializer.serialize(simulation.system))